## IESO

In [1]:
%load_ext autoreload
%autoreload 2

In [13]:
# # load from github
# import urllib.request
# api_url = 'https://raw.githubusercontent.com/tanmayyb/ele70_bv03/refs/heads/main/api/datasets.py'
# exec(urllib.request.urlopen(api_url).read())

# load from local
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(project_root)
from api.datasets import IESODataset

In [ ]:
dataset = IESODataset('fsa')
dataset.load_dataset(download=False)
dataset.selected_dates

## Climate

In [ ]:
# class WeatherDataset(Dataset):
#   def __init__(self, region: str = "ON", sampling_seed: int = 42, sample_num: int = 10):
#     super().__init__(region, "CANADA")
#     self.dataset_type = "WEATHER"
    
#     # add zonal list
#     # add sampling based on zonal list

In [ ]:
# class DatasetLoader():
#   def __init__(self, dataset: Dataset):
#     self.dataset = dataset

#   def load_dataset(self, dataset, start_date: int = None, end_date: int = None, download: bool = False):
#     if start_date > end_date:
#       raise ValueError("Start date must be before end date")

#     files = dataset.files
#     file_type = dataset.filetype
#     dates = dataset.dates
#     date_type = dataset.date_type

#     if download:
#       self.dataset.download_dataset()
#     else:
#       self.dataset.load_dataset_from_local()

  # def download_dataset(self, dataset, file_type, date_type, start_date: int, end_date: int):
  # # make sure data folder exists
  #   if file_type == "zip":
  #     self.download_zip_dataset(files, dates, date_type)
  #   elif file_type == "csv":
  #     self.download_csv_dataset(files, dates, date_type)

  # def download_zip_dataset(self, files, dates, date_type):
  #   for file in files:
  #     url = file